In [14]:
from __future__ import print_function
import fitz, sys

def split_pdf(finput):
    doc = fitz.open(finput)
    page = doc.loadPage(0)
    if int(page.MediaBox[2]) > 800.0:
        res = fitz.open() 

        for spage in doc:
            r = spage.rect  
            d = fitz.Rect(spage.CropBoxPosition, spage.CropBoxPosition)  
    
            r1 = fitz.Rect(0, 0, r.width*0.5, r.height)
            r2 = fitz.Rect(r.width*0.5, 0, r.width, r.height)
            rect_list = [r1, r2]
    
            for rx in rect_list:  
                rx += d 
                page = res.newPage(-1, width = rx.width, height = rx.height)
                page.showPDFpage(page.rect, doc, spage.number, clip = rx)

        res.save(doc.name[:-3]+'_sp.pdf', garbage = 3, deflate = True)
        return doc.name[:-3]+'_sp.pdf'
    else:
        return finput

In [15]:
import pdftotext
import os

def text22text(finput):
    
    fout = os.path.join(os.getcwd(), 'output.txt')
    
    if '.pdf' in finput:
         
        with open(finput, "rb") as f:
            pdf = pdftotext.PDF(f)

        with open('output.txt', 'w',-1,'utf-8') as f:
            f.write("\n\n".join(pdf))
    
        with open('output.txt', 'r',-1,'utf-8') as f:
            text=f.read()
            
    elif '.HWP' or '.hwp' in finput:
        hwp2txt_cmd = r'hwp5txt --output={} {}'.format(fout, finput)
        os.system(hwp2txt_cmd)
        with open(fout,'r',encoding='UTF8') as f:
            text=f.read()
            
    return text

In [16]:
import glob
from PIL import Image
from PyPDF2 import PdfFileReader
from pdf2image import convert_from_bytes
from pathos.multiprocessing import ProcessingPool as Pool

def image22text(finput, seqnum):
    img_name = os.path.join(r'C:\pgm_file', 'img_file', seqnum) 
    if '.hwp' in finput:
        return 0
    
    if not os.path.exists(img_name):
        os.mkdir(img_name)
        images = convert_from_bytes(open(finput, 'rb').read(), dpi=300, output_folder = img_name, fmt="png")
    
    img_list = list()
    for i in range(len(os.listdir(img_name))):
        img_list.append(img_name)
        
    with Pool(10) as p:
        text = p.map(image_multi, img_list, os.listdir(img_name))
    return " ".join(text)

def image_multi(img_name, img):
    import cv2
    import os
    import pytesseract
    img = cv2.imread(os.path.join(img_name, img))
    img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)

    custom_config = r'-l kor -c tessedit_char_blacklist=ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 6'
    temp = pytesseract.image_to_string(thresh, config = custom_config, lang='kor')
    return temp

In [17]:
import os
import pandas as pd 

tlist, ilist = list(), list()
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#1_file_list'}.xlsx")
df = pd.read_excel(xname, index_col=0)

for i in range(len(df)):
    
    finput = df.loc[i, 'path']
    print(finput)
    if 'pdf' in finput:
        finput = split_pdf(finput)

    text = text22text(finput)
    if text.isspace():
        text = image22text(finput, str(df.loc[i, 'seqnum']))
        ilist.append('1')
    else:
        ilist.append('0')
    tlist.append(text)
    
    print(i)
df['text'] = tlist
df['image'] = ilist
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df.to_excel(x2name)

C:\pgm_file\[형사] 특정범죄 가중처벌 등에 관한 법률(2016. 1. 6. 법률 제13717호로 개정된 것) 제5조의4 제5항 제1호 중 형법 제329조 부분에 대하여 위헌법률심판을 제청한 사건(의정부지방법원 2020노985).pdf
0
C:\pgm_file\[형사] 인근 토지 소유자의 원룸 신축공사로 인하여 자신 소유 토지들에 대한 사용권이 침해당한다는 이유로 공사부지 진입로에 철제 펜스를 설치한 피고인에 대하여 일반교통방해죄 및 업무방해죄로 벌금 200만 원을 선고한 사건(2019고정1947).pdf
1
C:\pgm_file\[형사]코로나19 상황을 이용하여 마스크를 팔겠다며 인터넷 사기범행을 저지른 피고인에게 실형을 선고한 사례(서울북부지방법원 2020고단1349).pdf
2
C:\pgm_file\[형사]술에 취해 식당에서 자신이 코로나19에 걸렸다고 소리치며 가래침을 뱉는 등의 행위를 한 피고인에게 집행유예(사회봉사 포함) 등을 선고한 사례(서울북부지방법원 2020고단1347).pdf
3
C:\pgm_file\[형사]아파트 각 층을 돌며 현관문 앞에 놓인 택배상자를 훔친 피고인에게 벌금형을 선고한 사례(서울북부지방법원 2020고단603).pdf
4
C:\pgm_file\[형사]1회 30만 원의 대가를 받고 금융위원회 직원을 사칭하여 보이스피싱 범죄에 가담한 피고인에게 실형을 선고한 사례(서울북부지방법원 2019고단4983).pdf
5
C:\pgm_file\[형사]대출을 돕겠다며 피해자의 신분증과 공인인증서 등을 받은 뒤 피해자 명의로 대출을 받아 이를 가로챈 피고인에게 실형을 선고한 사례(서울북부지방법원 2019고단4726).pdf
6
C:\pgm_file\[형사]재물손괴(전주지방법원 2019고단483).pdf
7
C:\pgm_file\[형사] 산림훼손 실형(울산지방법원 2019고합148).pdf
8
C:\pgm_file\[형사] 마세라티 견인 절도 사건(울산지방법원 2019고단2768).pdf
9
C:\pgm_file\[형사] 소개팅 젓갈,

122
C:\pgm_file\[형사] 특수강도미수 및 사기 등 사건{울산지방법원 2019고합264, 2019고합270(병합)}.pdf
123
C:\pgm_file\[형사] 사문서위조, 위조사문서행사 사건{울산지방법원 2019고단1026, 1246(병합)}.pdf
124
C:\pgm_file\[형사] 국내 거주 외국인들의 강도상해, 특수감금 등 사건{울산지방법원 2019고합129, 151-1(병합, 분리), 164(병합), 204(병합), 230(병합), 249(병합)}.pdf
125
C:\pgm_file\[형사]손가락 인대가 손상되었음에도 단순 피부 봉합술만 시행하여 피해자에게 손가락 굴곡변형이 발생하게 한 피고인에게 벌금 1,000만 원을 선고한 사례 (부산지방법원 2018고정1428).pdf
126
C:\pgm_file\[형사]의료과실로 피해자 2명을 사망에 이르게 한 피고인에게 금고 1년 3월에 집행유예 2년을 선고한 사례 [부산지방법원 2019고단2637, 4359(병합)].pdf
127
C:\pgm_file\[형사-국민참여재판] 특수상해(서울동부지방법원2019고합126).pdf
128
C:\pgm_file\[형사] 학부모로부터 금품받은 고교 야구부 감독에 대하여 항소심에서 추징금을 증액한 사례(대구지법 2019노2328).pdf
129
C:\pgm_file\[형사] 정보통신망이용촉진및정보보호등에관한법률위반(음란물유포)방조 판결.pdf
130
C:\pgm_file\[형사]클럽에서 상해를 가한 미군인에게 벌금 300만 원을 선고한 사례 (부산지방법원 2019고정1407).pdf
131
C:\pgm_file\[형사]친분이 있는 공무원을 통해 구청의 관급자재 수의계약을 알선하고 그 대가로 약 1억 원 상당의 이익을 취득한 피고인에게 징역 1년 및 추징을 선고한 사례 (부산지방법원 2019고합512).pdf
132
C:\pgm_file\[형사]프로축구 선수에게 금품을 제공할 의사를 표시하며 승부조작을 의뢰한 피고인에게 징역 10월을 선고한 사례 (부산지방법원 

219
C:\pgm_file\[형사] 이혼한 전 부인에 대한 살인미수 등 사건(울산지방법원 2019고합95).pdf
220
C:\pgm_file\[형사] 특정경제범죄가중처벌등에관한법률위반(횡령), 업무상횡령 및 사립학교법위반 등으로 기소된 피고인(前 신한대학교 총장)에게 징역 2년을 선고하고 법정 구속한 사안(의정부지방법원 2019. 11. 8. 선고 2018고합173 사건).pdf
221
C:\pgm_file\[형사] 만성 조현병 등으로 사물을 변별하거나 의사를 결정할 능력이 미약한 상태에서 피해자에게 상해를 가하여 사망에 이르게 한 피고인에게 실형 등을 선고한 사례(서울북부지방법원 2019고합167).pdf
222
C:\pgm_file\[형사] 민사판결 등에서 패소하자 자신의 억울한 사정을 언론에 알리기 위하여 도봉구청을 방화하려고 마음먹고 위 건조물을 소훼하려고 하였으나, 소속 공무원들에 의해 진화되어 미수에 그친 피고인이 직접 119와 112에 신고하고, 화재가 조기에 진압되어 인명피해가 발생하지 않고, 물적피해도 경미한 점 등을 이유로 집행유예 등을 선고한 사례(서울북부지방법원 2019고합117).pdf
223
C:\pgm_file\[형사] 피해자에게 중고 금 거래로 인한 수익금을 지급하여 줄 의사나 능력이 없으면서, 피해자를 기망하여 이에 속은 피해자로부터 금원을 편취한 피고인에게 실형을 선고한 사례(서울북부지방법원 2019고합17).pdf
224
C:\pgm_file\[형사] 약국개설자가 아니면서 성명불상자로부터 의약품을 판매할 목적으로 취득하고, 성명불상자와 공모하여 이를 판매한 피고인에게 집행유예를 선고한 사례(서울북부지방법원 2019고단1424).pdf
225
C:\pgm_file\[형사] 상호불상의 PC방에서, 인터넷 도박 사이트에 접속하여 우연한 승부에 의해 승패가 결정되는 속칭 ‘바카라’라는 도박을 상습적으로 한 피고인에게 집행유예를 선고한 사례(서울북부지방법원 2019고단1200).pdf
226
C:\pgm_file\[형사] 버스의 운

320
C:\pgm_file\[형사] 채팅앱을 통해 다른 남성의 사진을 보내면서 피해자의 호감을 얻고 피해자로부터 돈을 편취한 피고인에게 징역형을 선고한 사례(대구지법 2019고단3202).pdf
321
C:\pgm_file\[형사] 폭염경보가 발효된 날 옥외 공사현장에서 용접 작업을 하는 근로자를 보호할 의무를 위반한 관리책임자에게 징역형을 선고한 사례(대구지법 19고단3023).pdf
322
C:\pgm_file\[형사] 현행범 체포 과정에서 임의제출 압수(형사소송법 제218조)는 허용되지 않고, 휴대전화기가 임의제출된 것이라도 휴대전화기에 기억된 저장정보에 대한 탐색과 증거추출이 진행된다면, 참여권 보장과 관련성 요건과 같은 적법절차원칙은 지켜져야 한다는 이유로 피고인에 대하여 무죄판결을 선고한 사안(의정부지방법원 2019. 8. 22. 선고 2018노2757 판결).pdf
323
C:\pgm_file\[형사] 공무집행방해(서울동부지방법원 2019고단321).pdf
324
C:\pgm_file\[형사] 살인미수(서울동부지방법원 2019고합30).pdf
325
C:\pgm_file\[형사] 정보통신망이용촉진및정보보호등에관한법률위반 등(서울동부지방법원 2019고단1680).pdf
326
C:\pgm_file\[형사] 산업용지 소유자는 공장설립 완료신고 후 5년 이내에 처분할 수 없음에도 제3자에게 지분을 양도한 피고인에게 징역형을 선고한 사건(안산지원 2018고단4748).pdf
327
C:\pgm_file\[형사] 공무원인 피고인이 형사사법정보 누설하는 등 부당한 목적으로 사용한 것에 대하여 형의 선고를 유예한 사건(평택지원 2018고정194).pdf
328
C:\pgm_file\[형사] 피고인(목사)이 신도들을 대상으로 ‘타작마당’이라는 폭행, 상해 범행을 저지르고 아동학대를 범한 것에 대하여 중형을 선고한 사건(안양지원 2018고단1316).pdf
329
C:\pgm_file\[형사] 공장을 경매받은 피해자가 추후 환매요구를 거부하자 공장을 방화한 피고

418
C:\pgm_file\[형사] 피고인이 서울 소재 특급호텔 로비에서 난동을 부리고, 호텔 보안요원을 폭행하였으며, 신고를 받고 출동한 경찰관도 폭행하고 현행범인으로 체포된 상태에서 순찰차까지 발로 차는 등으로 손상한 사건{울산지방법원 2018고단2795, 2796(병합)}.pdf
419
C:\pgm_file\[형사] 공직선거법위반, 업무방해 사건(울산지방법원 2018고합288).pdf
420
C:\pgm_file\[형사] 공직선거법위반 사건(울산지방법원 2018고합251).pdf
421
C:\pgm_file\[형사] 공직선거법위반 사건(울산지방법원 2018고합280).pdf
422
C:\pgm_file\[형사] 쌍방폭행에서 서로 공격할 의사로 싸우면서 폭행을 가한 경우 정당방위에 해당하지 아니한다고 판단한 사례(대구지법 2018노4402).pdf
423
C:\pgm_file\[형사] 음주운전으로 피해자를 충격하여 사망에 이르게 한 사안(창원지방법원 2019고합19호).pdf
424
C:\pgm_file\[형사] 피고인이 스마트폰 어플 채팅방에서 만난 피해자(유부녀)와 성관계하는 장면을 몰래 녹화한 후 이를 이용하여 피해자를 협박하여 피해자로부터 1,600만 원을 갈취하고, 중학교 동창생인 또 다른 피해자를 기망하여 1,500만 원을 편취한 사건{울산지방법원 2018고단3805, 2019고단403(병합)}.pdf
425
C:\pgm_file\[형사] 업무상횡령, 근로기준법위반, 사기{울산지방법원 2018고단3116, 3850(병합), 2019고단3(병합)}.pdf
426
C:\pgm_file\사] 형사보상 및 명예회복에 관한 법률 제26조 제1항이 형사소송법에 따라 재심절차에서 원판결의 형보다 경한 형으로 유죄의 선고를 받아 확정되었으나 그 재심절차에서 공소장의 교환적 변경이 이루어지지 않았다면 무죄재판을 받을 만한 현저한 사유가 있었을 경우”를 형사보상 대상으로 규정하고 있지 않은 것은, 이른바 ‘부진정 입법 부작위’로서 입법재량의 한계를 벗어나 헌법 

...

6055
C:\pgm_file\[형사]500원권 상당의 물품을 구입하고 10,000원권 지폐를 지급하여 9,500원의 거스름돈을 받은 후 거스름돈으로 받은 5,000원권 지폐를 숨긴 채 거스름돈을 덜 받았다고 거짓말하여 다시 5,000원을 지급받는 수법으로 모두 6회에 걸쳐 30,000원을 편취하고, 2회에 걸쳐 10,000원을 편취하려다 미수에 그친 사안에서, 같은 수법의 동종전과가 수회 있는 점 등을 고려하여 징역 8월의 실형을 선고한 1심 판결.pdf
6056
C:\pgm_file\[형사]음주운전을 하였다는 공소사실에 대하여, 검찰이 제시한 증거만으로는 유죄를 인정하기에 부족하고, 오히려 여러 가지 사정에 비추어 피고인의 차를 대리운전해 준 기사가 허위신고를 하였을 가능성도 있어 보인다고 판시하면서 피고인에 대하여 무죄를 선고한 판결.pdf
6057
C:\pgm_file\[형사]사소한 문제로 동료 경비원의 멱살을 잡고 흔들다가 동료 경비원이 뿌리치는 바람에 뒤로 넘어지면서 손목을 다쳤음에도 동료 경비원으로부터 막대기 등으로 맞아 전치 12주의 상해를 입었다는 취지로 동료 경비원을 무고한 피고인에 대하여 1심은 유죄를 인정하면서 징역 6월에 집행유예 2년을 선고하였고, 이에 대하여 피고인이 사실오인의 주장을 하면서 항소하였으나 1심의 판단이 정확하다면서 피고인의 항소를 기각한 항소심 판결.pdf
6058
C:\pgm_file\[형사]식칼 등으로 공동상해의 범행을 저지르고, 체포 당시 경찰관에게 공무집행방해와 상해의 범행까지 저지른 캄보디아 국적의 불법체류자에 대하여 실형 2년을 선고한 1심을 유지한 항소심 판결.pdf
6059
C:\pgm_file\[형사]국민참여재판으로 진행된 성폭력범죄의처벌및피해자보호등에관한법률위반(주거침임강간등)사건에서 징역 5년의 실형을 선고하고, 피고인에 대한 열람정보를 5년간의 열람에 제공하고, 2년간 위치추적전자장치의 부착을 명한 판결.pdf
6060
C:\pgm_file\[형사]전국택시산업노동조합 부산지역본부 간부 3명이 학자

6103
C:\pgm_file\관할관청(시장, 군수, 구청장)의 승인 없이 차량 상판에 경광등과 앰프장치가 설치된 승합차량으로 항만순찰업무를 수행하는 바람에 부산광역시가 그 사용자로서 자동차관리법위반죄(관할관청의 승인 없이 구조 등이 변경된 자동차를 운행한 자의 사용자)로 기소된 사안에서, 위 공무원이 행한 사무는 부산남항 내 시설에 대한 순찰, 노숙자단속, 주차단속 등의 업무로서 부산광역시장이 국가로부터 위임받은 기관위임사무에 해당하고, 이러한 경우 부산.pdf
6104
C:\pgm_file\업발전법에 의한 대규모 점포개설자의 업무를 수행할 수 있는 지위만을 인정받은 회사가 명확한 법적 근거 없이 상가의 구분소유자들로부터 관리비를 징수하려고 하자 이에 항의하는 과정에서 공동감금, 업무방해, 모욕 등의 범행을 저지른 피고인들에 대하여, 피고인들이 이 사건 범행에 이르게 된 동기에 참작할 점이 있고, 피고인들이 입은 피해도 적지 않은 점 등 여러 가지 사정을 고려하여 벌금형을 선고한 원심을 파기하고 벌금형의 선고유예를.pdf
6105
C:\pgm_file\운용에 따른 명령을 위반한 행위를 처벌하는 새마을금고법 제85조 제2항은, 범죄의 구성요건에 해당하는 내용이 시행령에 포괄적으로 위임되어 있을 뿐만 아니라 시행령에서도 범죄의 구성요건을 국가기관도 아닌 단체에 위임하고 있는 것이어서 헌법 제13조 제1항(형벌불소급, 일사부재리의 원칙), 제37조 제2항(국민의 자유와 권리의 제한)에 반하여 무효이고, 양벌규정인 새마을금고법 제86조는 책임주의에 반하여 헌법에 위반된다고 판단하면.pdf
6106
C:\pgm_file\[형사]음주 및 무면허운전혐의로 기소된 피고인에 대한 심리 도중 진범이 피고인의 성명을 모용한 사실이 밝혀져 피고인에 대한 공소가 기각된 판결.pdf
6107
C:\pgm_file\[형사]특수절도죄 등으로 수회 처벌받은 전력이 있는 피고인이 가석방된 지 불과 5개월 만에 다시 상습절도 등의 범행을 저지른 사안에서, 피고인의 아들도 절도범행으로 처벌받고

6157
C:\pgm_file\[형사]유흥주점에서 발생한 화재로 모두 9명이 사망한 형사사건의 판결.pdf
6158
C:\pgm_file\[형사]공범의 정식재판청구사건(벌금형사건)에서 위증을 하여 위증죄로 불구속 기소되었음에도 계속하여 범행을 부인하면서 자신의 잘못을 뉘우치지 못하는 피고인에 대하여 징역 6월의 실형을 선고하고 법정구속한 판결.pdf
6159
C:\pgm_file\[형사]참기름에 옥수수기름을 섞어 가짜참기름을 제조하여 판매한 피고인들에 대한 형사판결.pdf
6160
C:\pgm_file\[민사]무죄판결은 공소사실에 대한 입증이 없다는 의미일 뿐 공소사실의 부존재가 증명되었다는 의미가 아니라고 판단하면서, 형사소송에서 이미 무죄판결이 확정되었음에도 불구하고 관련 민사소송에서는 원고의 청구를 인용한 판결.pdf
6161
C:\pgm_file\[형사]폭우가 내리는 야간에 술에 취하여 편도 4차로를 무단횡단 하던 피해자를 충격하여 사망에 이르게 한 피고인에 대하여, 주의의무위반(사고당시의 상황에 비추어 속도를 50%줄여 운전할 의무)의 잘못을 인정하여 유죄를 인정하면서, 합의가 이루어진 점 등의 사정을 고려하여 금고형의 집행유예를 선고한 판결.pdf
6162
C:\pgm_file\[형사]가족들의 신고로 체포된 피고인에 대하여 본드흡입의 습벽을 고치는데 필요한 기간 등을 감안하여 징역 1년의 실형을 선고한 판결.pdf
6163
C:\pgm_file\[형사]종교적인 문제로 병역을 거부한 피고인에 대하여 1년 6월의 형을 선고한 판결.pdf
6164
C:\pgm_file\[형사]동종범죄로 집행유예기간 중인 것을 비롯하여 동종범죄전력이 모두 11회에 있는 피고인이 또다시 2회에 걸친 무면허운전을 한 사안에서 징역 6월의 실형을 선고하면서 법정구속한 판결.pdf
6165
C:\pgm_file\[형사]사종교상의 이유로 병역거부를 한 피고인에 대하여 징역 1년 6월을 선고한 판결.pdf
6166
C:\pgm_file\[형사]경미한 교통사고를 당하고도 장기간 입원하

6263
C:\pgm_file\[형사] 임신 7월의 태아를 낙태한 의사에게 징역형(6월, 집행유예 1년) 및 자격정지(1년) 형을 선고한 사례.pdf
6264
C:\pgm_file\[형사] 간헐적이고 협박의 정도가 약한 문자전송은 정보통신망이용촉진및정보보호등에관한법률위반이 아니라고 본 사례.pdf
6265
C:\pgm_file\[형사] 상습적으로 어린 자녀를 학대한 부부에게 중형(3년, 2년)을 선고한 사례.pdf
6266
C:\pgm_file\[형사] 환경단체 회원을 가장하여 금품을 갈취한 사례.pdf
6267
C:\pgm_file\[형사] 임금체불 사업자가 근로자의 합의서를 위조하였다가 실형(10월)을 선고받은 사례.pdf
6268
C:\pgm_file\[형사] 건축허가를 받고자 가공의 마을주민단체를 작출한 사례.pdf
6269
C:\pgm_file\[형사] 시각장애인이 아닌 정상인의 안마사 영업을 처벌하는 것이 부당하다는 피고인의 변소를 배척한 사례.pdf
6270
C:\pgm_file\[형사] TV로 범행수법을 익혀 강도상해에 이르게 된 사례.pdf
6271
C:\pgm_file\태안앞바다에서 발생한 유조선기름유출사고와 관련하여 대전지방법원에서 2008. 12. 10.선고된 형사항소심 판결문 입니다.pdf
6272
C:\pgm_file\형사사건에 있어서 공소사실의 특정 정도.pdf
6273
C:\pgm_file\[형사] 부산시의회 청사 증축공사 비리사건.pdf
6274
C:\pgm_file\[형사] 도로공사 직원의 수뢰사건.pdf
6275
C:\pgm_file\[형사] 2명을 살해하고, 1명을 중태에 빠뜨린 사안에서 무기징역형을 선고한 사례.pdf
6276
C:\pgm_file\[형사] 버릇을 고친다고 중학교로 찾아가 중학생들을 폭행한 사례.pdf
6277
C:\pgm_file\[형사] 새마을금고 이사장이 금고보유 주식을 시세보다 낮게 매수한 사안에서 업무상배임죄를 적용한 사례.pdf
6278
C:\pgm_file\[형사] 경리여직원이 회사자금을 횡령하

6376
C:\pgm_file\[형사] 통장 개설 등의 방법으로 보이스피싱 범죄에 협조한 사건(강릉지원 2007노253).pdf
6377
C:\pgm_file\[형사] 간통 및 명예훼손으로 기소된 사건(강릉지원 2006노412).pdf
6378
C:\pgm_file\[형사] 학원의설립ㆍ운영및과외교습에관한법률위반--무죄.pdf
6379
C:\pgm_file\[형사] 양도한 현금카드가 보이스피싱범죄에 사용된 사건(원주지원 2007고단506).pdf
6380
C:\pgm_file\[형사]  경찰 및 검찰에서 자백한 후 법정에서 범행 부인하고, 피해자들이 성명불상자로부터 절도를 당하였다는 진술만 있는 경우 유죄 인정 여부.pdf
6381
C:\pgm_file\특수공무집행방해 등 형사사건.pdf
6382
C:\pgm_file\[형사] 불법체류 및 전자금융거래 접근매체를 양도한 사건(원주지원 2007고단497).pdf
6383
C:\pgm_file\[형사] 자동차를 폐차장에 무단방치하여 기소된 사건(강릉지원 2007노63).pdf
6384
C:\pgm_file\[형사] 위법한 임의동행에 이은 음주측정요구에 응할 의무가 있는 지 여부(강릉지원 2006노329).pdf
6385
C:\pgm_file\[형사] 평소 사이가 좋지않음을 이유로 도로 통행을 막은 사건(원주지원 2007고정381).pdf
6386
C:\pgm_file\[형사] 노래방에서 주류 판매 및 접대부 알선한 사건(원주지원 2007고정325).pdf
6387
C:\pgm_file\[형사] 아파트주차구역 내의 통로부분이 도로교통법상의 도로에 해당하는 지 여부(원주지원 2007고정178).pdf
6388
C:\pgm_file\[형사] 교통사고를 가장하여 사망보험금을 편취하기 위해 살인을 교사한 사건(원주지원 2006고합63).pdf
6389
C:\pgm_file\[형사] 회사의 보통예금 통장에서 돈을 인출하여 생활비로 사용한 사건(강릉지원 2006고단254).pdf
6390
C:\pgm_file\[형사] 하천부지를